# Data Cleaning

In [64]:
# Imports
import numpy as np, pandas as pd

In [65]:
# Read in data set: CCES 2010-14 panel
df_cces = pd.read_stata('../Data/CCES/CCES_Panel_Full3waves_VV_V4_LRS.dta')

In [66]:
# Have a brief look at the data set ==> Wide format
df_cces

,caseid,weight,CC10_301_1,CC10_301_2,CC10_301_3,CC10_301_4,CC10_301_5,CC10_301b,CC10_301c,CC10_302,...,validated_reg_10,validated_reg_12,validated_reg_14,VV_general_10,VV_general_12,VV_general_14,VV_primary_10,VV_primary_12,VV_primary_14,VV_pres_primary_12
0,25233.0,0.538062,Yes,Yes,Yes,Yes,No,Both,Print,Gotten much worse,...,Registered,Registered,No match found,In person or unknown method,In person,Unmatched,Voted in Primary,Voted in Primary,Unmatched,No primary vote record
1,38716.0,3.470877,Yes,No,Yes,Yes,No,NaN,Both,Stayed about the same,...,Registered,Registered,Registered,Mail voter,Mail voter,Mail voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
2,7796.0,0.602477,No,Yes,Yes,Yes,No,Both,Print,Gotten worse,...,Registered,Registered,Registered,Early voter,Early voter,Early voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
3,78113.0,0.264208,Yes,Yes,Yes,Yes,No,Both,Online,Gotten better,...,Registered,Registered,Registered,Absentee voter,Absentee voter,Absentee voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
4,63800.0,0.196632,Yes,Yes,Yes,No,No,Both,Both,Gotten better,...,Registered,Registered,Registered,Early voter,In person,In person,Did not vote in primary,Voted in Primary,Voted in Primary,No primary vote record
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,57290.0,1.146715,No,No,Yes,Yes,No,NaN,Online,Gotten better,...,Registered,Registered,Registered,Confirmed Non-voter,Voted by unknown method,Voted by unknown method,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9496,57315.0,1.171217,No,No,No,Yes,No,NaN,NaN,Stayed about the same,...,Registered,Registered,Registered,In person or unknown method,In person,Confirmed Non-voter,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9497,57399.0,0.424949,No,Yes,Yes,No,No,Both,Print,Gotten better,...,On multiple registration lists,Registered,Registered,Confirmed Non-voter,In person,Confirmed Non-voter,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9498,59331.0,0.505942,No,Yes,No,Yes,No,Local Newscast,NaN,Gotten worse,...,Registered,Registered,Registered,In person or unknown method,In person,In person,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record


In [67]:
# Convert caseid to integer
df_cces['caseid'] = df_cces['caseid'].astype(int)

In [68]:
# Inspect column names
for col in df_cces.columns:
    print(col)

caseid
weight
CC10_301_1
CC10_301_2
CC10_301_3
CC10_301_4
CC10_301_5
CC10_301b
CC10_301c
CC10_302
CC10_304
CC10_305
CC10_308a
CC10_308b
CC10_308c
CC10_308d
CC10_308e
CC10_309a
CC10_309b
CC10_309c
CC10_309d
CC10_310a
CC10_310b
CC10_310c
CC10_310d
CC10_315a
CC10_315b
CC10_315c
CC10_316
CC10_317
CC10_317_t
CC10_320
CC10_321
CC10_322_1
CC10_322_2
CC10_322_3
CC10_322_4
CC10_322_7
CC10_322_8
CC10_324
CC10_325
CC10_326
CC10_327
CC10_328
CC10_329
CC10_330A
CC10_330B
CC10_330C
CC10_330D
CC10_330E
CC10_330F
CC10_330G
CC10_330H
CC10_330I
CC10_330J
CC10_334E
CC10_334Hb
CC10_334Ib
CC10_335a1
CC10_335a2
CC10_335b1
CC10_335b2
CC10_335c1
CC10_335c2
CC10_341A
CC10_341B
CC10_341C
CC10_341F
CC10_341G
CC10_341H
CC10_341I
CC10_341J
CC10_341K
CC10_341L
CC10_341M
CC10_341R
CC10_350
CC10_351
CC10_352a
CC10_352b
CC10_354
CC10_355
CC10_355_t
CC10_355a
CC10_355a_t
CC10_355b
CC10_355b_t
CC10_356
CC10_356_t
CC10_356a
CC10_356a_t
CC10_390
CC10_390_t
CC10_390a
CC10_390a_t
CC10_401
CC10_402a
CC10_402a_t
CC10_402b
CC1

In [69]:
# Discard open questions
cols_open = [col for col in df_cces.columns if col.endswith('_t') or '_t_' in col or col.endswith('_other')]
print(f'{len(cols_open)} open questions detected.')
df_cces.drop(cols_open, axis=1, inplace=True)

151 open questions detected.


In [70]:
# Discard contextual variables
patterns = [
    'CurrentGov', 'CurrentHouse', 'CurrentSen1', 'CurrentSen2', 'GovCand', 'HouseCand',
    'HouseSeat', 'LegName', 'LowerChamberName', 'OtherGovCand', 'OtherHouseCand',
    'SenCand', 'FormerHouse', 'HouseVote', 'Senate1Vote', 'Senate2Vote', 'Ohousecan2party',
    'Otherhousecandidate2party', 'Rsenatecandidategender'
]
cols_context = []
for col in df_cces.columns:
    for pattern in patterns:
        if pattern in col:
            cols_context.append(col)
print(f'{len(cols_context)} contextual variables detected.')
df_cces.drop(cols_context, axis=1, inplace=True)

380 contextual variables detected.


In [71]:
# Check
for col in df_cces.columns:
        print(col)

caseid
weight
CC10_301_1
CC10_301_2
CC10_301_3
CC10_301_4
CC10_301_5
CC10_301b
CC10_301c
CC10_302
CC10_304
CC10_305
CC10_308a
CC10_308b
CC10_308c
CC10_308d
CC10_308e
CC10_309a
CC10_309b
CC10_309c
CC10_309d
CC10_310a
CC10_310b
CC10_310c
CC10_310d
CC10_315a
CC10_315b
CC10_315c
CC10_316
CC10_317
CC10_320
CC10_321
CC10_322_1
CC10_322_2
CC10_322_3
CC10_322_4
CC10_322_7
CC10_322_8
CC10_324
CC10_325
CC10_326
CC10_327
CC10_328
CC10_329
CC10_330A
CC10_330B
CC10_330C
CC10_330D
CC10_330E
CC10_330F
CC10_330G
CC10_330H
CC10_330I
CC10_330J
CC10_334E
CC10_334Hb
CC10_334Ib
CC10_335a1
CC10_335a2
CC10_335b1
CC10_335b2
CC10_335c1
CC10_335c2
CC10_341A
CC10_341B
CC10_341C
CC10_341F
CC10_341G
CC10_341H
CC10_341I
CC10_341J
CC10_341K
CC10_341L
CC10_341M
CC10_341R
CC10_350
CC10_351
CC10_352a
CC10_352b
CC10_354
CC10_355
CC10_355a
CC10_355b
CC10_356
CC10_356a
CC10_390
CC10_390a
CC10_401
CC10_402a
CC10_402b
CC10_403
CC10_405
CC10_406a
CC10_406b_1
CC10_406b_2
CC10_406b_3
CC10_406c
CC10_410b
CC10_410b_nv
CC10_411
C

In [72]:
# Look at unusual column names
for col in df_cces.columns:
    if not col.startswith('CC') and not col.endswith(('_10', '_12', '_14')):
        print(col)

caseid
weight
V202_wave2010
V292_post_wave2010
inputzip
inputzip_post
inputzip_wave2012
investor_14_a
matched
partyaffiliation
sourcerec_post
state_cl


In [73]:
# Omit variables not mentioned in codebook which are not of interest
df_cces.drop(['state_cl', 'sourcerec_post', 'V202_wave2010', 'V292_post_wave2010', 'inputzip',
              'inputzip_post', 'inputzip_wave2012', 'investor_14_a', 'matched', 'partyaffiliation'], axis=1, inplace=True)

In [74]:
# Deal with district identifiers
df_cces['cdid112_10'] = df_cces['cdid112_10'].astype(int)
df_cces['cdid113_12'] = df_cces['cdid113_12'].astype(int)
df_cces['cdid114_14'] = df_cces['cdid114_14'].astype(int)

In [75]:
# Check # of discrepancies in district id between pre- and post-election survey
print('2010:', np.sum(df_cces['cdid112_10'] != df_cces['cdid112_post_10'].astype(int)))
print('2012:', np.sum(df_cces['cdid113_12'] != df_cces['cdid113_post_12'].astype(int)))
print('2014:', np.sum(df_cces['cdid114_14'] != df_cces['cdid114_post_14'].astype(int)))

2010: 109
2012: 23
2014: 36


In [76]:
# Rename cd ids such that we get one common variable in long format
df_cces.rename({'cdid112_10': 'cdid_10',
                'cdid113_12': 'cdid_12',
                'cdid114_14': 'cdid_14'}, axis=1, inplace=True)

In [77]:
# Get columns per year
# 2010
cols_2010 = [col for col in df_cces.columns if col.startswith('CC10_') or col.endswith('_10')]
assert len(cols_2010) == len(set(cols_2010)) # ensure that colnames are unique

In [78]:
# Check
for col in cols_2010:
    if col.startswith(('CC12', 'CC14')) or col.endswith(('_12', '_14')):
        print(col)

CC12_387_10
CC12_399_10
CC12_418bx_10
CC14_387_10
CC14_418bx_10


In [79]:
# Omit columns from other waves accidentally ending up in 2010 subset
cols_2010 = [col for col in cols_2010 if not col.startswith(('CC12', 'CC14'))]

In [80]:
# 2012
cols_2012 = [col for col in df_cces.columns if col.startswith('CC12_') or col.endswith('_12')]
assert len(cols_2012) == len(set(cols_2012)) # ensure that colnames are unique

In [81]:
# Check
for col in cols_2012:
    if col.startswith(('CC10', 'CC14')) or col.endswith(('_10', '_14')):
        print(col)

CC12_387_10
CC12_399_10
CC12_418bx_10
CC14_387_12


In [82]:
# Omit column from other wave accidentally ending up in 2012 subset
cols_2012.remove('CC14_387_12')

In [83]:
# 2014
cols_2014 = [col for col in df_cces.columns if col.startswith('CC14_') or col.endswith('_14')]
assert len(cols_2014) == len(set(cols_2014)) # ensure that colnames are unique

In [84]:
# Check
for col in cols_2014:
    if col.startswith(('CC10', 'CC12')) or col.endswith(('_10', '_12')):
        print(col)

CC14_387_10
CC14_387_12
CC14_418bx_10


In [85]:
df_cces

,caseid,weight,CC10_301_1,CC10_301_2,CC10_301_3,CC10_301_4,CC10_301_5,CC10_301b,CC10_301c,CC10_302,...,validated_reg_10,validated_reg_12,validated_reg_14,VV_general_10,VV_general_12,VV_general_14,VV_primary_10,VV_primary_12,VV_primary_14,VV_pres_primary_12
0,25233,0.538062,Yes,Yes,Yes,Yes,No,Both,Print,Gotten much worse,...,Registered,Registered,No match found,In person or unknown method,In person,Unmatched,Voted in Primary,Voted in Primary,Unmatched,No primary vote record
1,38716,3.470877,Yes,No,Yes,Yes,No,NaN,Both,Stayed about the same,...,Registered,Registered,Registered,Mail voter,Mail voter,Mail voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
2,7796,0.602477,No,Yes,Yes,Yes,No,Both,Print,Gotten worse,...,Registered,Registered,Registered,Early voter,Early voter,Early voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
3,78113,0.264208,Yes,Yes,Yes,Yes,No,Both,Online,Gotten better,...,Registered,Registered,Registered,Absentee voter,Absentee voter,Absentee voter,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record
4,63800,0.196632,Yes,Yes,Yes,No,No,Both,Both,Gotten better,...,Registered,Registered,Registered,Early voter,In person,In person,Did not vote in primary,Voted in Primary,Voted in Primary,No primary vote record
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,57290,1.146715,No,No,Yes,Yes,No,NaN,Online,Gotten better,...,Registered,Registered,Registered,Confirmed Non-voter,Voted by unknown method,Voted by unknown method,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9496,57315,1.171217,No,No,No,Yes,No,NaN,NaN,Stayed about the same,...,Registered,Registered,Registered,In person or unknown method,In person,Confirmed Non-voter,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9497,57399,0.424949,No,Yes,Yes,No,No,Both,Print,Gotten better,...,On multiple registration lists,Registered,Registered,Confirmed Non-voter,In person,Confirmed Non-voter,Did not vote in primary,Did not vote in primary,Did not vote in primary,No primary vote record
9498,59331,0.505942,No,Yes,No,Yes,No,Local Newscast,NaN,Gotten worse,...,Registered,Registered,Registered,In person or unknown method,In person,In person,Voted in Primary,Voted in Primary,Voted in Primary,No primary vote record


In [86]:
# Write a function to get rootnames (i.e., without CC10, _10)
def get_root_names(col_list: list, yy_end):
    root_names = []
    for col in col_list:
        if col.startswith('CC') and col.endswith(yy_end):
            split = col.split('_')
            root_name = '_'.join(split[1:])
            root_names.append(root_name)
        elif col.startswith('CC'):
            split = col.split('_')
            root_name = '_'.join(split[1:])
            root_names.append(root_name)
        elif col.endswith(yy_end):
            split = col.split('_')
            root_name = '_'.join(split[:-1])
            root_names.append(root_name)
        else:
            root_names.append(col)
            
    return root_names

In [87]:
cols_2010 = set(get_root_names(cols_2010, '_10'))
cols_2012 = set(get_root_names(cols_2012, '_12'))
cols_2014 = set(get_root_names(cols_2014, '_14'))

In [88]:
# In 2012 & 2014 there are a lot more vars compared to 2010
# However, the vast majority of the additional vars in 2012
# and 2014 are not of main interest for this project
len(cols_2010), len(cols_2012), len(cols_2014)

(246, 426, 443)

In [89]:
# Get common vars across all three waves
cols_common = cols_2010.intersection(cols_2012, cols_2014)
print(f'There are {len(cols_common)} common variables across all three waves.')

There are 223 common variables across all three waves.


In [90]:
# Exclude 
# - unvalidated registration & turnout (intention 354); except turnout 2008
# - reasons for not voting & mode of voting
# - all columns containing response category "I did not vote"
cols_drop = [
    'regzip', 'regzip_post', 'StateAbbr', 'countyfips', 'countyfips_post',
    'countyname', 'countyname_post', 'votereg', 'regzip', 'regzip_post',
    'votereg_f', '352b', '354', '401', '402a', '402b', '403', '405', '410b',
    '410b_nv', '411', '411_nv', '412', '412_nv', '412a', '412b', '413a', '413b',
    '413c', '413d', 'reszip', 'reszip_post', 'starttime', 'starttime_post',
    'validated_reg', 'race_other', 'endtime', 'endtime_post'
]
for col in cols_drop:
    cols_common.discard(col)
    
print(f'Now we are left with {len(cols_common)} variables.')

Now we are left with 187 variables.


In [91]:
# Create subsets per wave including only the 187 common variables
cols_2010 = ['CC10_' + col if col[0].isdigit() else col + '_10' for col in cols_common]
cols_2010.sort()
assert len(cols_2010) == len(cols_common)
cols_2012 = ['CC12_' + col if col[0].isdigit() else col + '_12' for col in cols_common]
cols_2012.sort()
assert len(cols_2012) == len(cols_common)
cols_2014 = ['CC14_' + col if col[0].isdigit() else col + '_14' for col in cols_common]
cols_2014.sort()
assert len(cols_2014) == len(cols_common)

In [92]:
# Create new data frames per year
df_cces10 = df_cces[['caseid', 'weight'] + cols_2010].copy()
df_cces12 = df_cces[['caseid', 'weight'] + cols_2012].copy()
df_cces14 = df_cces[['caseid', 'weight'] + cols_2014].copy()
df_cces10.shape[1], df_cces12.shape[1], df_cces14.shape[1]

(189, 189, 189)

In [93]:
# Add variable for year in every newly created data frame
df_cces10.insert(2, 'year', 2010)
df_cces12.insert(2, 'year', 2012)
df_cces14.insert(2, 'year', 2014)

In [94]:
# Write function to rename the variables, similar to get_root_names
def rename_df(df: pd.DataFrame, yy_end):
    dic_rename = {}
    for col in df.columns:
        if col.startswith('CC') and col.endswith(yy_end):
            split = col.split('_')
            root_name = '_'.join(split[1:])
            dic_rename[col] = root_name
        elif col.startswith('CC'):
            split = col.split('_')
            root_name = '_'.join(split[1:])
            dic_rename[col] = root_name
        elif col.endswith(yy_end):
            split = col.split('_')
            root_name = '_'.join(split[:-1])
            dic_rename[col] = root_name
        else:
            dic_rename[col] = col
            
    df.rename(columns=dic_rename, inplace=True)

In [95]:
# Apply function to the dfs
rename_df(df_cces10, '_10')
rename_df(df_cces12, '_12')
rename_df(df_cces14, '_14')

In [96]:
# Merge data frames ==> long format
df_cces_clean = pd.concat([df_cces10, df_cces12, df_cces14]).reset_index(drop=True)

In [97]:
# Glimpse at new data frame in long format
df_cces_clean

,caseid,weight,year,301_1,301_2,301_3,301_4,301_5,301b,301c,...,religpew_mormon,religpew_muslim,religpew_nondenom,religpew_orthodox,religpew_pentecost,religpew_presby,religpew_protestant,religpew_reformed,union,unionhh
0,25233,0.538062,2010,Yes,Yes,Yes,Yes,No,Both,Print,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I formerly was a member of a labor union,"No, no one in my household has ever been a mem..."
1,38716,3.470877,2010,Yes,No,Yes,Yes,No,NaN,Both,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I am not now, nor have I been, a member of a l...","No, no one in my household has ever been a mem..."
2,7796,0.602477,2010,No,Yes,Yes,Yes,No,Both,Print,...,The Church of Jesus Christ of Latter-day Saints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I formerly was a member of a labor union,A member of my household was formerly a member...
3,78113,0.264208,2010,Yes,Yes,Yes,Yes,No,Both,Online,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I am not now, nor have I been, a member of a l...","No, no one in my household has ever been a mem..."
4,63800,0.196632,2010,Yes,Yes,Yes,No,No,Both,Both,...,NaN,NaN,NaN,NaN,NaN,NaN,Methodist,NaN,"I am not now, nor have I been, a member of a l...","No, no one in my household has ever been a mem..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28495,57290,1.146715,2014,No,No,No,No,Yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I am not now, nor have I been, a member of a l...",A member of my household was formerly a member...
28496,57315,1.171217,2014,No,No,No,No,Yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I am not now, nor have I been, a member of a l...","No, no one in my household has ever been a mem..."
28497,57399,0.424949,2014,No,Yes,No,No,No,Local Newscast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I am not now, nor have I been, a member of a l...",A member of my household was formerly a member...
28498,59331,0.505942,2014,No,Yes,No,Yes,No,Local Newscast,NaN,...,The Church of Jesus Christ of Latter-day Saints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, I am currently a member of a labor union","No, no one in my household has ever been a mem..."


In [98]:
# Replace empty strings with NaN
df_cces_clean.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_cces_clean.replace('', np.nan, inplace=True)

In [99]:
# Look at proportions of missing values per variable
for col, prop in (df_cces_clean.isna().mean()
                  .sort_values(ascending=False)
                  .items()):
    print(f"{col:<17} {prop:.2f}")

religpew_hindu    1.00
religpew_muslim   1.00
religpew_advent   1.00
religpew_holiness 1.00
406c              0.99
religpew_orthodox 0.99
religpew_reformed 0.99
religpew_buddhist 0.99
406b_3            0.99
406b_2            0.99
406b_1            0.99
religpew_mormon   0.98
religpew_christian 0.98
religpew_congreg  0.98
religpew_pentecost 0.97
religpew_episcop  0.97
religpew_presby   0.96
religpew_jewish   0.96
418bx_8           0.95
418bx_2           0.95
418bx_10          0.95
418bx_6           0.95
418bx_7           0.95
418bx_9           0.95
418bx_1           0.95
418bx_5           0.95
418bx_4           0.95
418bx_3           0.95
religpew_lutheran 0.94
religpew_methodist 0.92
religpew_nondenom 0.88
religpew_baptist  0.88
child18num        0.84
341M              0.82
423c              0.81
religpew_catholic 0.76
355               0.73
356               0.72
421_rep           0.69
421b              0.68
417c              0.66
390               0.64
417bx_8           0.64
417bx_1 

In [100]:
# Vast majority of columns with very high no of NaN are not of interest
# Discard all columns containing more than 24% NaN
thresh = 0.24
df_cces_clean.dropna(axis=1, thresh=int((1-thresh)*len(df_cces_clean)), inplace=True)
print(f'Now we are left with {df_cces_clean.shape[1]} variables.')

Now we are left with 130 variables.


In [101]:
# Look at proportions of missing values per variable
for col, prop in (df_cces_clean.isna().mean()
                  .sort_values(ascending=False)
                  .items()):
    print(f"{col:<17} {prop:.2f}")

citylength_2      0.23
425b_4            0.23
425b_1            0.23
425b_2            0.23
425b_3            0.23
301b              0.22
addrlength_2      0.22
406a              0.08
341K              0.07
hispanic          0.07
317               0.06
423a              0.05
341L              0.04
internetwork      0.04
330E              0.03
423b              0.03
addrlength_1      0.03
350               0.02
citylength_1      0.02
329               0.02
341R              0.02
330F              0.02
330C              0.02
315a              0.01
330H              0.01
328               0.01
330G              0.01
310d              0.01
330A              0.01
330B              0.01
315c              0.01
416r              0.01
315b              0.01
341F              0.01
341B              0.01
330D              0.01
324               0.01
unionhh           0.01
309c              0.01
309d              0.01
310b              0.01
326               0.01
310c              0.00
308e       

In [102]:
# Notice that the responses do not contain any NaN
for col, prop in (df_cces_clean.isna().mean()
                  .sort_values(ascending=False)
                  .items()):
    if col in ['VV_primary', 'VV_general']:
        print(f"{col:<17} {prop:.2f}")

VV_general        0.00
VV_primary        0.00


In [103]:
# Recode some of the columns, where answer categories are long
df_cces_clean['316'].replace({'Yes. I definitely voted.': 'Yes',
                              'I usually vote, but did not in 2008': 'Usually vote but no',
                              'I am not sure': 'Not sure'}, inplace=True)
df_cces_clean['321'].replace({
   'Global climate change has been established as a serious problem, and immediate action is necessary.': 'Immediate action',
   'There is enough evidence that climate change is taking place and some action should be taken.': 'Some action',
   'Concern about global climate change is exaggerated.  No action is necessary.': 'No action',
   "We don't know enough about global climate change, and more research is necessary before we take any actions.": 'More research',
   'Global climate change is not occurring; this is not a real issue.': 'Climate change is not occurring'
}, inplace=True)
df_cces_clean['324'].replace({
   'By law, a woman should always be able to obtain an abortion as a matter of personal choice': 'Always',
   "The law should permit abortion only in case of rape, incest or when the woman's life is in danger": 'Only rape, incest or life in danger',
   "The law should permit abortion for reasons other than rape, incest, or danger to the woman's life, but only after the ne": 'Not only rape, incest or life in danger',
   "By law, abortion should never be permitted": 'Never'
}, inplace=True)
df_cces_clean['325'].replace({
    'Economy somewhat more important': 'Economy more important',
    'Environment somewhat more important': 'Environment more important',
    'Much more important to protect environment even if we lose jobs or lower our standard of living': 'Environment much more important',
    'Much more important to protect jobs, even if the environment gets worse': 'Economy much more important',
    'Much more important to protect jobs, even if environment worse': 'Economy much more important',
    'Much more important to protect environment even if lose jobs and lower standard of living': 'Environment much more important'
}, inplace=True)
df_cces_clean['union'].replace({
    'I am not now, nor have I been, a member of a labor union': 'No, never',
    'I formerly was a member of a labor union': 'Formerly',
    'Yes, I am currently a member of a labor union': 'Yes, currently'
}, inplace=True)
df_cces_clean['unionhh'].replace({
    'No, no one in my household has ever been a member of a labor union': 'No, never',
    'A member of my household was formerly a member of a labor union, but is not now': 'Formerly',
    'Yes, a member of my household is currently a union member': 'Yes, currently'
}, inplace=True)

In [104]:
# Recode 415r and 416r
# Remove letters to convert to float
df_cces_clean['415r'].replace({'0- All from tax increases': '0',
                               '100- All from spending cuts': '100'}, inplace=True)
df_cces_clean['416r'].replace({'0- All from sales tax': '0',
                               '100- All from income taxes': '100'}, inplace=True)
# Convert to float
df_cces_clean['415r'] = df_cces_clean['415r'].astype(float)
df_cces_clean['416r'] = df_cces_clean['416r'].astype(float)

In [105]:
# Both variables contain the value 997 ==> NaN
np.max(df_cces_clean['415r']), np.max(df_cces_clean['416r'])

(997.0, 997.0)

In [106]:
df_cces_clean['415r'].replace(997.0, np.nan, inplace=True)
df_cces_clean['416r'].replace(997.0, np.nan, inplace=True)

In [107]:
# Replace $-sign of income variable with empty string and recode
df_cces_clean['faminc'] = df_cces_clean['faminc'].str.replace('$', '', regex=False)
df_cces_clean['faminc'].replace({'less than 10,000': 'Less than 10,000'}, inplace=True)

In [108]:
# Recode 315a, 315b and 315c which contain same category twice
for col in df_cces_clean.columns:
    if col.startswith('315'):
        df_cces_clean[col].replace(
            {'Never Heard': 'Never Heard of Person'}, inplace=True
        )

In [109]:
# Harmonize 330A-H between waves (support/oppose in 2010, yes/no in 2012/14)
for col in df_cces_clean.columns:
    if col.startswith('330'):
        df_cces_clean[col].replace({'Oppose': 'No', 'Support': 'Yes'}, inplace=True)

In [110]:
# Harmonize ideo5 between waves
df_cces_clean['ideo5'].replace({'Very Conservative': 'Very conservative'}, inplace=True)

In [111]:
# Recode the dependent variables to dummies. Mode of voting is not of our interest.
# Assume that unmatched = abstainers
df_cces_clean['VV_primary'].replace({'Voted in Primary': 'Voted',
                                     'Did not vote in primary': 'Abstained',
                                     'Unmatched': 'Abstained'}, inplace=True)
df_cces_clean['VV_general'].replace({'In person': 'Voted',
                                     'In person or unknown method': 'Voted',
                                     'Voted by unknown method': 'Voted',
                                     'Confirmed Non-voter': 'Abstained',
                                     'Unmatched': 'Abstained',
                                     'Absentee voter': 'Voted',
                                     'Early voter': 'Voted',
                                     'Mail voter': 'Voted'}, inplace=True)

In [112]:
df_cces_clean['VV_primary'].value_counts()

Abstained    15321
Voted        13179
Name: VV_primary, dtype: int64

In [113]:
df_cces_clean['VV_general'].value_counts()

Voted        21066
Abstained     7434
Name: VV_general, dtype: int64

In [114]:
# Look at dtypes of data frame
df_cces_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28500 entries, 0 to 28499
Columns: 130 entries, caseid to unionhh
dtypes: category(105), float64(7), int64(3), object(15)
memory usage: 8.3+ MB


In [115]:
# Convert birthyr categorical ==> int64
df_cces_clean['birthyr'] = df_cces_clean['birthyr'].astype(int)

In [116]:
# Convert object dtypes to categorical
for col in df_cces_clean.select_dtypes('object').columns:
    df_cces_clean[col] = df_cces_clean[col].astype('category')

In [117]:
# Convert all ordered categories to unordered categories
for col in df_cces_clean.columns:
    if pd.api.types.is_categorical_dtype(df_cces_clean[col]) and df_cces_clean[col].cat.ordered:
        df_cces_clean[col] = df_cces_clean[col].cat.as_unordered()

In [118]:
# Create ordered categories (only where meaningful / possible)
df_cces_clean['302'].replace({'Not sure': np.nan}, inplace=True)
df_cces_clean['302'] = df_cces_clean['302'].cat.set_categories(
    ['Gotten much worse', 'Gotten worse', 'Stayed about the same',
     'Gotten better', 'Gotten much better'], ordered=True
)
df_cces_clean['320'] = df_cces_clean['320'].cat.set_categories(
    ['Less Strict', 'Kept As They Are', 'More Strict'], ordered=True
)
df_cces_clean['321'] = df_cces_clean['321'].cat.set_categories(
    ['Climate change is not occurring', 'No action',
     'More research', 'Some action', 'Immediate action'], ordered=True
)
df_cces_clean['324'] = df_cces_clean['324'].cat.set_categories(
    ['Never', 'Only rape, incest or life in danger', 
     'Not only rape, incest or life in danger', 'Always'], ordered=True
)
df_cces_clean['325'].replace({'Not sure': np.nan}, inplace=True)
df_cces_clean['325'] = df_cces_clean['325'].cat.set_categories(
    ['Environment much more important', 'Environment more important',
     'About the same', 'Economy more important', 'Economy much more important'],
    ordered=True
)
df_cces_clean['327'] = df_cces_clean['327'].cat.set_categories(
    ['Strongly oppose', 'Somewhat oppose', 
     'Somewhat support', 'Strongly support'], ordered=True
)
df_cces_clean['educ'] = df_cces_clean['educ'].cat.set_categories(
    ['No HS', 'High school graduate', 'Some college', 
     '2-year', '4-year', 'Post-grad'], ordered=True
)
df_cces_clean['ideo5'].replace({'Not sure': np.nan}, inplace=True)
df_cces_clean['ideo5'] = df_cces_clean['ideo5'].cat.set_categories(
    ['Very liberal', 'Liberal', 'Moderate', 
     'Conservative', 'Very conservative'], ordered=True
)
df_cces_clean['newsint'].replace({"Don't know": np.nan}, inplace=True)
df_cces_clean['newsint'] = df_cces_clean['newsint'].cat.set_categories(
    ['Hardly at all', 'Only now and then', 
     'Some of the time', 'Most of the time'], ordered=True
)
df_cces_clean['pew_churatd'].replace({"Don't know": np.nan}, inplace=True)
df_cces_clean['pew_churatd'] = df_cces_clean['pew_churatd'].cat.set_categories(
    ['Never', 'Seldom', 'A few times a year', 'Once or twice a month', 
     'Once a week', 'More than once a week'], ordered=True
)
df_cces_clean['pew_prayer'].replace({"Don't know": np.nan}, inplace=True)
df_cces_clean['pew_prayer'] = df_cces_clean['pew_prayer'].cat.set_categories(
    ['Never', 'Seldom', 'A few times a month', 'Once a week',
    'A few times a week', 'Once a day', 'Several times a day'], ordered=True
)
df_cces_clean['pew_religimp'] = df_cces_clean['pew_religimp'].cat.set_categories(
    ['Not at all important', 'Not too important', 
     'Somewhat important', 'Very important'], ordered=True
)
df_cces_clean['pid7'].replace({'Not sure': np.nan}, inplace=True)
df_cces_clean['pid7'] = df_cces_clean['pid7'].cat.set_categories(
    ['Strong Democrat', 'Not very strong Democrat', 'Lean Democrat', 'Independent', 
     'Lean Republican', 'Not very strong Republican', 'Strong Republican'], ordered=True
)
for col in df_cces_clean.columns:
    if col.startswith('308'):
        df_cces_clean[col].replace({'Not sure': np.nan}, inplace=True)
        df_cces_clean[col] = df_cces_clean[col].cat.set_categories(
            ['Strongly Disapprove', 'Somewhat Disapprove',
             'Somewhat Approve', 'Strongly Approve'], ordered=True
        )
    elif col.startswith('341'):
        df_cces_clean[col].replace({'Not Sure': np.nan}, inplace=True)
        df_cces_clean[col] = df_cces_clean[col].cat.set_categories(
            ['Very Liberal', 'Liberal', 'Somewhat Liberal',
             'Middle of the Road', 'Somewhat Conservative',
             'Conservative', 'Very Conservative'], ordered=True
        )
    elif col.startswith('422'):
        df_cces_clean[col] = df_cces_clean[col].cat.set_categories(
            ['Strongly disagree', 'Somewhat disagree', 
             'Neither agree nor disagree',
             'Somewhat agree', 'Strongly agree'], ordered=True
        )

In [119]:
''' Unfortunately, the researchers used different income
categories across the 3 survey waves. Hence, we leave it
as a nominal variable for now. '''
df_cces_clean['faminc'].value_counts()

Prefer not to say    3359
80,000 - 99,999      2859
50,000 - 59,999      2604
40,000 - 49,999      2562
30,000 - 39,999      2467
70,000 - 79,999      2315
60,000 - 69,999      2216
100,000 - 119,999    2146
120,000 - 149,999    1786
20,000 - 29,999      1400
10,000 - 19,999       922
150,000 - 199,999     770
150,000 or more       626
Less than 10,000      569
25,000 - 29,999       411
20,000 - 24,999       361
200,000 - 249,999     304
15,000 - 19,999       271
10,000 - 14,999       226
250,000 - 349,999     179
350,000 - 499,999      62
500,000 or more        46
250,000 or more         5
Name: faminc, dtype: int64

In [120]:
''' Check number of instances where a respondent moved from one state
to another between the pre- and a post-election survey within a wave'''
np.sum(df_cces_clean['inputstate'] != df_cces_clean['inputstate_post'])

32

In [121]:
# Occurred only 32 times ==> omit inputstate_post
df_cces_clean.drop('inputstate_post', axis=1, inplace=True)

In [122]:
# Now some variables contain more than 24% missing values
for col, prop in (df_cces_clean.isna().mean()
                  .sort_values(ascending=False)
                  .items()):
    if prop > 0.24:
        print(f"{col:<17} {prop:.2f}")

341K              0.36
341L              0.31


In [123]:
# Omit those variables
df_cces_clean.drop(['341K', '341L'], axis=1, inplace=True)

In [124]:
""" Save cleaned data set as parquet, which is much more 
efficient than csv / stata and preserves dtypes """
#df_cces_clean.to_parquet('../Data/CCES/CCES_Panel_cleaned.parquet')

' Save cleaned data set as parquet, which is much more \nefficient than csv / stata and preserves dtypes '